In [2]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2

In [11]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame
        results = face_detection.process(rgb_frame)

        # Draw bounding boxes
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)

        # Display the output
        cv2.imshow("Face Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
# Set up directories
DATASET_PATH = "dataset"
PERSON_NAME = input("Enter person's name: ")  # Input person's name
SAVE_PATH = os.path.join(DATASET_PATH, PERSON_NAME)
os.makedirs(SAVE_PATH, exist_ok=True)

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

print("Capturing images... Press 'q' to stop.")

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        face_resized = cv2.resize(face, (224, 224))  # Resize to match model input size

        # Save image
        filename = os.path.join(SAVE_PATH, f"{count}.jpg")
        cv2.imwrite(filename, face_resized)
        count += 1

        # Draw rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Face Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f"Captured {count} images for {PERSON_NAME}.")
    

Capturing images... Press 'q' to stop.
Captured 631 images for kishan.


In [7]:
datagen = 'C:/Users/vishw/Desktop/face learn/dataset'


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    datagen,
    label_mode="int",  # Labels as integers (0 or 1)
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),  # Resize images for faster processing
    batch_size=32)
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    datagen,
    label_mode="int",  # Labels as integers (0 or 1)
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),  # Resize images for faster processing
    batch_size=32)

class_names = train_dataset.class_names
print("Class names:", class_names)

# Data Augmentation (only for training)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.01),
    tf.keras.layers.RandomZoom(0.05)
])


normalization_layer = tf.keras.layers.Rescaling(1./255)


train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)  
)
train_dataset = train_dataset.map(
    lambda x, y: (normalization_layer(x), y) 
)

validation_dataset = validation_dataset.map(
    lambda x, y: (normalization_layer(x), y)
)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Create CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(len(class_names), activation='softmax')  
  
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, validation_data=validation_dataset, epochs=2, batch_size=32)

# Save the trained model
model.save("face_recognition_model.h5")

print("Model training complete and saved.")


Found 1402 files belonging to 3 classes.
Using 1122 files for training.
Found 1402 files belonging to 3 classes.
Using 280 files for validation.
Class names: ['kishan', 'nayab', 'vishwajith']
Epoch 1/2
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 766ms/step - accuracy: 0.8996 - loss: 0.2770 - val_accuracy: 0.3714 - val_loss: 4.4000
Epoch 2/2
36/36 ━━━━━━━━━━━━━━━━━━━━ 29s 798ms/step - accuracy: 0.9954 - loss: 0.0353 - val_accuracy: 0.4143 - val_loss: 7.1201


Model training complete and saved.


In [10]:
model = tf.keras.models.load_model("face_recognition_model.h5")

# Load class labels
DATA_DIR = "dataset"
class_names = sorted(os.listdir(DATA_DIR))  # Ensure classes match dataset folder names

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        face = cv2.resize(face, (224, 224))  # Resize to model input size
        face = face / 255.0  # Normalize
        face = np.expand_dims(face, axis=0)  # Add batch dimension

        # Predict class
        predictions = model.predict(face)
        class_id = np.argmax(predictions)
        class_label = class_names[class_id]

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, class_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display frame
    cv2.imshow("Live Facial Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━